<a href="https://colab.research.google.com/github/Sinabehzadifard/CyptoMarketPrediction/blob/main/Crypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-binance
!pip install mpl-finance
import matplotlib.pyplot as plt
!pip install prophet
from prophet import Prophet

In [ ]:
import pandas as pd
import numpy as np

import time
import dateparser
import pytz
import json

import math  

import datetime as dt
from datetime import datetime, timedelta

from threading import Thread

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.dates as mdates

from mpl_finance import candlestick_ochl

import datetime
from datetime import datetime


from sklearn import preprocessing, model_selection, neighbors, svm
from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error

from tqdm import tqdm as tqdm

from binance.client import Client

In [ ]:
def binanceDataFrame(klines):
  df = pd.DataFrame(klines.reshape(-1,12), dtype=float, columns=['Open Time',
                                                                 'Open',
                                                                 'High',
                                                                 'Low',
                                                                 'Close',
                                                                 'Volume',
                                                                 'Close Time',
                                                                 'Quote asset volume',
                                                                 'Number of trades',
                                                                 'Taker buy base asset volume',
                                                                 'Taker buy quote asset volume',
                                                                 'Can be ignored']);

  df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
  df['Close Time'] = pd.to_datetime(df['Close Time'], unit='ms')

  return df

In [ ]:
"Binance API"
api_key = 'aHWF2puEzrzj4vaaPmvIFGW0rUI1FPr4dw26orntUulPA466mY8KRW9xzeF5iveJ'
api_secret = ''
client = Client(api_key, api_secret)

In [ ]:
depth = client.get_order_book(symbol = 'BTCUSDT')

# Get BTCUSDT 1DAY from Binance

In [ ]:
%%time
klines = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1HOUR, "1 March, 2021")
klines = np.array(klines)
df = binanceDataFrame(klines)

In [ ]:
df.describe()

In [ ]:
remove_cols = [c for c in df.columns if c not in ['Close',"Open Time"	]]
df.drop(remove_cols, axis=1, inplace=True)

In [ ]:
df

In [ ]:
df.columns = ['ds', 'y']
df

In [ ]:
# define the model
model = Prophet( changepoint_prior_scale = 0.75)
# fit the model
model.fit(df)

In [ ]:
future = model.make_future_dataframe(periods=240, freq='H')
forecast = model.predict(future)
fig1 = model.plot(forecast)

In [ ]:
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(model, forecast)

In [29]:
plot_components_plotly(model, forecast)